Header

In [2]:
from vaccination_forecaster_io import *
import datetime
import math
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt

In [3]:
path = "../data/covid_vaccine.xlsx"
initial_table = pd.read_excel(path, engine="openpyxl")
cleaned_table = clean_excel_table(initial_table)


In [4]:
def generate_daily_differences(panda_data_frame_column, column_name):
	'''
	This function will take an input column and compute the differences 
	between each row and output another column
	'''
	inital_list = [panda_data_frame_column[0]]

	for index, row in enumerate(panda_data_frame_column.items()):
		if index > 0:
			inital_list.append(
				panda_data_frame_column[index]-panda_data_frame_column[index-1])

	column = pd.Series(inital_list)
	column.name = column_name
	return column


In [5]:
daily_first_vaccines = generate_daily_differences(
	cleaned_table.people_vaccinated, "daily_first_vaccines")
daily_second_vaccines = generate_daily_differences(
    cleaned_table.people_fully_vaccinated, "daily_second_vaccines")


NameError: name 'clean_table' is not defined